![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Disponibilización de modelos

En este notebook aprenderá a guardar un modelo y a disponibilizarlo como una API con la librería Flask. Una API (interfaz de programación de aplicaciones) es un conjunto de definiciones y protocolos que permiten que servicios, en este caso modelos, retornen resultados y respuestas sin necesidad de saber cómo están implementados.

## Instrucciones Generales:

Este notebook esta compuesto por dos secciones. En la primera secciónn, usted beberá entrenar y guardar (exportar) un modelo de random forest para predecir si una URL es phishing (fraudulenta) o no. En la segunda parte, usará el modelo entrenado y lo disponibilizara usando la libreria *Flask*. En el siguente paper puede conocer más detalles de la base de datos que usaremos y del problema: *A. Correa Bahnsen, E. C. Bohorquez, S. Villegas, J. Vargas, and F. A. Gonzalez, “Classifying phishing urls using recurrent neural networks,” in Electronic Crime Research (eCrime), 2017 APWG Symposium on. IEEE, 2017, pp. 1–8*. https://albahnsen.files.wordpress.com/2018/05/classifying-phishing-urls-using-recurrent-neural-networks_cameraready.pdf
  
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 


## Importar base de datos y librerías

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import joblib

import os
os.chdir('..')

In [3]:
# Carga de datos de archivos .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/phishing.csv')
data.head()

,url,phishing
0,http://www.subalipack.com/contact/images/sampl...,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1
2,http://theotheragency.com/confirmer/confirmer-...,1
3,http://aaalandscaping.com/components/com_smart...,1
4,http://paypal.com.confirm-key-21107316126168.s...,1


## Codificar variables categóricas
Relizar preprocesamiento de texto (URLs) para crear variables predictoras:

In [4]:
# Creación de columnas binarias que indican si la URL contiene la palabra clave (keywords)
keywords = ['https', 'login', '.php', '.html', '@', 'sign']
for keyword in keywords:
    data['keyword_' + keyword] = data.url.str.contains(keyword).astype(int)

# Definición de la variable largo de la URL
data['lenght'] = data.url.str.len() - 2

# Definición de la variable largo del dominio de la URL
domain = data.url.str.split('/', expand=True).iloc[:, 2]
data['lenght_domain'] = domain.str.len()

# Definición de la variable binaria que indica si es IP
data['isIP'] = (domain.str.replace('.', '') * 1).str.isnumeric().astype(int)

# Definicón de la variable cuenta de 'com' en la URL
data['count_com'] = data.url.str.count('com')

data.head()

,url,phishing,keyword_https,keyword_login,keyword_.php,keyword_.html,keyword_@,keyword_sign,lenght,lenght_domain,isIP,count_com
0,http://www.subalipack.com/contact/images/sampl...,1,0,0,0,0,0,0,47,18,0,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1,0,0,0,0,0,0,73,41,0,0
2,http://theotheragency.com/confirmer/confirmer-...,1,0,0,0,0,0,0,92,18,0,1
3,http://aaalandscaping.com/components/com_smart...,1,0,0,0,0,0,0,172,18,0,3
4,http://paypal.com.confirm-key-21107316126168.s...,1,0,0,0,0,0,0,90,50,0,1


In [5]:
# Separación de variables predictoras (X) y variable de interes (y)
X = data.drop(['url', 'phishing'], axis=1)
y = data.phishing

## Entrenar y guardar el modelo

In [6]:
# Definición de modelo de clasificación Random Forest
clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=3)
cross_val_score(clf, X, y, cv=10)

array([0.7515 , 0.7655 , 0.75425, 0.755  , 0.74775, 0.76575, 0.7565 ,
       0.76325, 0.75725, 0.75725])

In [7]:
# Entrenamiento del modelo de clasificación Random Forest
clf.fit(X, y)

RandomForestClassifier(max_depth=3, n_jobs=-1)

In [8]:
# Exportar modelo a archivo binario .pkl

joblib.dump(clf, 'model_deployment/phishing_clf.pkl', compress=3)

['model_deployment/phishing_clf.pkl']

In [9]:
# Importar modelo y predicción
from model_deployment.m09_model_deployment import predict_proba

# Predicción de probabilidad de que un link sea phishing
predict_proba('http://www.vipturismolondres.com/com.br/?atendimento=Cliente&/LgSgkszm64/B8aNzHa8Aj.php')

0.7479906126618306

## Disponibilizar modelo con Flask

Para esta sección del notebook instale las siguientes librerías *!pip install flask* y *!pip install flask_restplus*.

In [31]:
pip install flask-restplus

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install --upgrade werkzeug

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install Werkzeug==2.2.3

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [33]:
conda list werkzeug

# packages in environment at /Users/cristianfabianpelayocarrillo/opt/anaconda3:
#
# Name                    Version                   Build  Channel
werkzeug                  1.0.1              pyhd3eb1b0_0  

Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install flask_restx

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.8 MB 1.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [38]:
# Importación librerías

from flask import Flask
from flask_restx import Api, Resource, fields

In [39]:
# Definición aplicación Flask
app = Flask(__name__)

# Definición API Flask
api = Api( # Aqui están diciendo que es una API
    app, 
    version='1.0', 
    title='Phishing Prediction API',
    description='Phishing Prediction API')

ns = api.namespace('predict', 
     description='Phishing Classifier')

# Definición argumentos o parámetros de la API
parser = api.parser()
parser.add_argument(
    'URL', # Que le va a llegar un argumento 
    type=str, #De tipo string.
    required=True, 
    help='URL to be analyzed', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [40]:
# Definición de la clase para disponibilización

#Aquí es como va a procesar ese resultado. 

@ns.route('/')
class PhishingApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args() #Va a coger el string que le va a llegar 
        
        return {
         "result": predict_proba(args['URL'])# y lo manda a la función predicr_prob
        }, 200

In [ ]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.11:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Apr/2023 13:56:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 13:56:47] "GET /swaggerui/swagger-ui.css HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 13:56:47] "GET /swaggerui/droid-sans.css HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 13:56:47] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 13:56:47] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 13:56:48] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 13:56:48] "GET /swaggerui/favicon-16x16.png HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 13:56:48] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 13:58:15] "GET /predict/?URL=http%3A%2F%2Fconsultoriojuridico.co%2Fpp%2Fwww.paypal.com%2F, HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 14:00:15] "GET /predict/?URL=http://cons

El modelo debe haber quedado disponibilizado en el puerto 5000. Para predecir la probabilidad de que una URL sea fraudulenta (phishing) copie en la barra de busqueda de su navegador la siguiente dirección (http://localhost:5000/predict/?URL=) y agregregue al final de esta la URL que desee precir. Por ejemplo, al copiar la URL http://localhost:5000/predict/?URL=http://consultoriojuridico.co/pp/www.paypal.com/, la API retornará la probabilidad de que la URL http://consultoriojuridico.co/pp/www.paypal.com/ sea phishing.